In [6]:
import json
import csv
import os
import pandas as pd
import numpy as np
import random
import copy
from datetime import datetime

path = './dataset/'

## Test 데이터 생성

In [7]:

# JSON 파일 경로
json_file_path = path + 'review.json'

# CSV 파일 경로
csv_file_path = path + 'checkin.csv'

# CSV 파일을 쓰기 모드로 열기
with open(csv_file_path, 'w', newline='') as csv_file:
    # CSV 라이터 생성
    csv_writer = csv.writer(csv_file)

    # CSV 파일 헤더 작성
    csv_writer.writerow(['user_id', 'business_id'])

    # JSON 파일을 한 줄씩 읽어서 처리
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            data = json.loads(line)  # JSON 데이터 파싱
            # user_id와 business_id 추출
            user_id = data['user_id']
            business_id = data['business_id']

            # CSV 파일에 데이터 작성
            csv_writer.writerow([user_id, business_id])

print("checkin_data 생성 완료.")

checkin_data 생성 완료.


In [8]:
# CSV 파일 경로
input_file = path + 'checkin.csv'
output_file = path +  'checkin_data_Yelp.csv'

# CSV 파일을 읽고 데이터를 리스트로 저장
data = []
with open(input_file, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)  # 헤더 행
    #data.append(header)
    for row in csv_reader:
        data.append(row)

# user_id를 기준으로 데이터를 정렬
data.sort(key=lambda x: x[0])  # 여기서 0은 user_id 열을 가리킵니다. 0부터 시작하면 첫 번째 열입니다.

# user_id를 정수형으로 변환
idx = 0
before_user_id = data[0][0]
for i in data:
  if i[0] == before_user_id:
    i[0] = idx
  else:
    idx += 1
    before_user_id = i[0]
    i[0] = idx

# business_id 기준으로 데이터를 정렬
data.sort(key=lambda x: x[1])  # 여기서 0은 user_id 열을 가리킵니다. 0부터 시작하면 첫 번째 열입니다.
print(data[:3])

# business_id 정수형으로 변환
idx = 0
before_business_id = data[0][1]
for i in data:
  if i[1] == before_business_id:
    i[1] = idx
  else:
    idx += 1
    before_business_id = i[1]
    i[1] = idx

print(data[:3])

# 다시 유저 번호 기준으로 정렬
data.sort(key=lambda x: x[0])
print(data[:3])

# 정렬된 데이터를 새로운 파일에 저장
with open(output_file, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(data)

print("CSV 파일이 user_id를 기준으로 정렬되었고, 새로운 파일에 저장되었습니다.")

[[156830, '---kPU91CF4Lq2-WlRu9Lw'], [209756, '---kPU91CF4Lq2-WlRu9Lw'], [213557, '---kPU91CF4Lq2-WlRu9Lw']]
[[156830, 0], [209756, 0], [213557, 0]]
[[0, 100781], [1, 3783], [1, 5256]]
CSV 파일이 user_id를 기준으로 정렬되었고, 새로운 파일에 저장되었습니다.


In [9]:
# 파일로부터 데이터를 읽기
# 이 코드에서는 파일 경로 'checkin_data.txt'로 가정합니다.
df = pd.read_csv(path + 'checkin_data_Yelp.csv', header=None, names=['ID', 'Value'])

# 각 ID의 출현 빈도를 계산
frequency = df['ID'].value_counts()

# 출현 빈도가 10 이상인 ID만 필터링
valid_ids = frequency[frequency >= 10].index

# 필터링된 ID에 해당하는 데이터만 선택
filtered_df = df[df['ID'].isin(valid_ids)]

# 결과를 새 파일로 저장하거나 출력
filtered_df.to_csv(path + 'filtered_data.csv', index=False, header=False)
# 혹은 출력하여 확인
print(filtered_df)

              ID   Value
1              1    3783
2              1    5256
3              1    5256
4              1    5256
5              1   10914
...          ...     ...
6990271  1987924   96954
6990272  1987924   98187
6990273  1987924  107385
6990274  1987924  120353
6990275  1987924  127628

[3303811 rows x 2 columns]


In [10]:
# 파일로부터 데이터를 읽기
df = pd.read_csv(path + 'filtered_data.csv', header=None, names=['ID', 'Value'])

# 고유한 ID를 정렬된 리스트로 추출합니다.
unique_ids = sorted(df['ID'].unique())

# 고유 ID에 대해 0부터 시작하는 새로운 인덱스를 매핑합니다.
id_mapping = {id: index for index, id in enumerate(unique_ids)}

# 이 매핑을 사용하여 모든 ID 값을 업데이트합니다.
df['ID'] = df['ID'].map(id_mapping)

# 결과 확인
print(df)

# 결과를 파일로 저장할 수 있습니다.
df.to_csv(path + 'reindexed_data.csv', index=False, header=False)


             ID   Value
0             0    3783
1             0    5256
2             0    5256
3             0    5256
4             0   10914
...         ...     ...
3303806  117369   96954
3303807  117369   98187
3303808  117369  107385
3303809  117369  120353
3303810  117369  127628

[3303811 rows x 2 columns]


In [11]:
# 파일로부터 데이터를 읽기
df = pd.read_csv(path + 'reindexed_data.csv', header=None, names=['ID', 'Value'])

# 고유한 ID를 정렬된 리스트로 추출합니다.
unique_ids = sorted(df['Value'].unique())

# 고유 ID에 대해 0부터 시작하는 새로운 인덱스를 매핑합니다.
id_mapping = {id: index for index, id in enumerate(unique_ids)}

# 이 매핑을 사용하여 모든 ID 값을 업데이트합니다.
df['Value'] = df['Value'].map(id_mapping)

# 결과 확인
print(df)

df = df.drop_duplicates(subset=['ID', 'Value'])

# 결과를 파일로 저장할 수 있습니다.
df.to_csv(path + 'pos.csv', index=False, header=False)


             ID   Value
0             0    3655
1             0    5073
2             0    5073
3             0    5073
4             0   10518
...         ...     ...
3303806  117369   93529
3303807  117369   94718
3303808  117369  103572
3303809  117369  116066
3303810  117369  123074

[3303811 rows x 2 columns]


In [12]:
# 데이터 로딩
data = pd.read_csv(path + 'pos.csv', header=None, names=['user_id', 'item_id'])

# 중복 제거
# drop_duplicates 함수를 사용하여, 각 사용자 ID와 아이템 ID 조합의 첫 번째 등장만 유지합니다.
data = data.drop_duplicates(subset=['user_id', 'item_id'])

# 각 사용자별로 첫 번째 데이터를 테스트 데이터로, 나머지를 트레이닝 데이터로 분리
train_data = []
test_data = []

# 사용자별로 그룹화하여 처리
for user_id, group in data.groupby('user_id'):
    test_data.append(group.iloc[0].tolist())  # 첫 번째 행을 테스트 데이터에 추가, 리스트로 변환
    train_data.extend(group.iloc[1:].values.tolist())  # 나머지 행을 트레이닝 데이터에 추가, 리스트로 변환

# 리스트를 DataFrame으로 변환
train_df = pd.DataFrame(train_data, columns=['user_id', 'item_id'])
test_df = pd.DataFrame(test_data, columns=['user_id', 'item_id'])

# CSV 파일로 저장
train_df.to_csv(path + 'train.csv', index=False, header=False)
test_df.to_csv(path + 'test.csv', index=False, header=False)

print("Files saved: 'train.csv' and 'test.csv'")


Files saved: 'train.csv' and 'test.csv'


In [11]:
# 필요한 파일을 읽고 쓰기 위해 open 함수를 사용
with open(path + 'neg_100.txt', 'r') as neg_file, open(path + 'test.csv', 'r') as test_file, open(path + 'test_neg_100.csv', 'w') as result_file:
    # 파일에서 각 줄을 읽음
    neg_lines = neg_file.readlines()
    test_lines = test_file.readlines()
    
    # neg_lines와 test_lines의 길이가 같다고 가정하고, 각 줄을 합침
    for neg_line, test_line in zip(neg_lines, test_lines):
        # test_line에서 데이터를 추출하여 형식을 맞춤
        test_index, test_value = test_line.strip().split(',')
        test_tuple = f'({test_index},{test_value})'
        
        # neg_line에서 쉼표를 탭으로 변환
        neg_data = '\t'.join(neg_line.strip().split(','))
        
        # 결과를 파일에 쓰기
        result_line = test_tuple + '\t' + neg_data.strip() + '\n'
        result_file.write(result_line)

# 결과 파일인 'result.txt'가 생성되고 원하는 형태로 데이터가 저장됨


In [ ]:
# 데이터 파일 로드
df = pd.read_csv('pos.txt', header=None, names=['user', 'item'])

# 각 사용자가 가진 아이템의 집합을 만듭니다.
user_items = df.groupby('user')['item'].apply(set)

# 전체 아이템 집합
all_items = set(df['item'])

# 각 사용자별로 가지고 있지 않은 아이템을 찾아 랜덤하게 100개 선택
results = {}
for user, items in user_items.items():
    not_owned_items = list(all_items - items)
    if len(not_owned_items) > 100:
        selected_items = np.random.choice(not_owned_items, 100, replace=False)
    else:
        selected_items = not_owned_items
    results[user] = selected_items

# 결과를 파일로 저장, 사용자 ID 없이 아이템만 저장
with open('neg_100.txt', 'w') as file:
    for items in results.values():
        items_str = ','.join(map(str, items))
        file.write(f"{items_str}\n")
